## 1. Studying Decorators

https://3months.tistory.com/344 참고

In [7]:
def a_new_decorator(a_func):
    # 함수 안에 함수를 정의하고 함수를 리턴한다
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

# 이 함수를 decoration (기능을 추가) 하고 싶음
def a_function_requiring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")

In [8]:
a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
a_function_requiring_decoration

<function __main__.a_new_decorator.<locals>.wrapTheFunction()>

In [9]:
@a_new_decorator
def a_function_requiring_decoration_2():
    """Hey you! Decorate me!"""
    print("I am the function which needs some decoration to" "remove my foul smell")
    
a_function_requiring_decoration_2()

I am doing some boring work before executing a_func()
I am the function which needs some decoration toremove my foul smell
I am doing some boring work after executing a_func()


In [11]:
# 근데 함수명이 이상하게 나옴. wrapTheFunction
print(a_function_requiring_decoration_2.__name__)

wrapTheFunction


- @ 키워드를 통해 a_function_requiring_decoration을 재정의하지 않아도 된다. 
- @ [함수명] 을 decoration하고 싶은 함수 위에 붙여주면 된다.
- 근데 함수 명이 wrapTheFunction으로 decoration한 함수의 이름이 그대로 나오게 된다. 
- 이를 해결하기 위해 wraps를 이용한다.

In [13]:
# wraps를 이용해 함수명이 제대로 나오게 할 수 있음 
from functools import wraps

# 최종적인 decorator의 일반적인 형태 
# a_function_requiring_decoration을 a_new_decorator로 decorating한다는 것이다. 
# 이 때 decorate할 함수는 a_func에 지정하고 이를 wraps로 받아서 그 아래 함수로 decoration함

In [17]:
def a_new_decorator2(a_func):
    @wraps(a_func)
    def wrapTheFunction(): 
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

@a_new_decorator2
def a_function_requiring_decoration3():
    """Hey you! Decorate me!"""
    print("I am the function which needs some decoration to" "remove my foul smell")
    
a_function_requiring_decoration3()

I am doing some boring work before executing a_func()
I am the function which needs some decoration toremove my foul smell
I am doing some boring work after executing a_func()


In [18]:
print(a_function_requiring_decoration3.__name__)

a_function_requiring_decoration3


- 이렇게 decorator에 wraps를 붙여주면, 그 함수를 decoration해주는 함수로 인식을 하게 된다. 
- 함수명도 기존의 함수 명인 a_function_requiring_decoration을 따르게 된다. 

### Decoration 활용의 좋은 예 - Authentication

authentication_check라는 함수를 한 번 만들면 그 다음부터 @authentication_check만 붙이면 저절로 검사해줌 (autentication - function 실행 순으로 알아서 함)

In [19]:
# decorator 함수
def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated

### 파이썬 자료구조와 알고리즘 실습: p. 142

In [20]:
import random
import time

def benchmark(func):
    def wrapper(*args, **kwargs):
        t = time.perf_counter()
        res = func(*args, **kwargs)
        print("{0} {1}".format(func.__name__, time.perf_counter() - t))
        return res
    return wrapper


@benchmark
def random_tree(n):
    temp = [n for n in range(n)]
    for i in range(n+1):
        temp[random.choice(temp)] = random.choice(temp)
    return temp

if __name__ == '__main__':
    random_tree(10000)
    

random_tree 0.027042100000016944


In [21]:
random_tree(10)

random_tree 2.4900000425986946e-05


[8, 1, 8, 3, 4, 8, 8, 3, 4, 8]

In [22]:
if __name__ == '__main__':
    random_tree(10)

random_tree 2.320000021427404e-05


In [24]:
__name__

'__main__'

In [23]:
random_tree.__name__

'wrapper'

In [25]:
random_tree(12)

random_tree 2.27999998969608e-05


[3, 1, 9, 6, 9, 5, 2, 6, 2, 9, 6, 6]

여기서 의문점: if __name__ = '__main__'로 함수를 불렀을 땐 res값이 안 나오지만, 그냥 random_tree 함수를 바로 부르면 res값도 함께 출력된다!왜 그럴까

## 2. @classmethod와 @staticmethod의 차이점

참고: https://stackoverflow.com/questions/136097/difference-between-staticmethod-and-classmethod, 파이썬 자료구조와 알고리즘 p.142

In [33]:
class A(object):
    def foo(self, x):
        print("executing foo({0}, {1})".format(self,x))
    
    @classmethod
    def class_foo(cls, x):
        print("executing class_foo({0}, {1})".format(cls,x))
        
    @staticmethod
    def static_foo(x):
        print("executing static_foo({0})".format(x))

a = A()

### Usual way an object instance calls a method: the object instance, a, is implicitly passed as the first argument

In [34]:
a

In [35]:
a.foo(10)

executing foo(<__main__.A object at 0x0000022952878988>, 10)


### With classmethods, the class of the object instance is implicitly passed as the first argument instead of self

In [36]:
A.foo(10)

TypeError: foo() missing 1 required positional argument: 'x'

In [37]:
A.class_foo(10)

executing class_foo(<class '__main__.A'>, 10)


You can also call class_foo using the class. In fact, if you define something to be a classmethod, it is probably because you intend to call it from the class rather than from a class instance. A.foo(1) would have raised a TypeError, but A.class_foo(1) works fine

In [42]:
a.class_foo(10)

executing class_foo(<class '__main__.A'>, 10)


### With staticmethods, neither self (the object instance) nor cls (the class) is implicitly passed as the first argument. They behave like plain functions except that you can call them from an instance or the class:

In [38]:
a.static_foo(1)

executing static_foo(1)


In [39]:
A.static_foo('hi')

executing static_foo(hi)


In [40]:
a.static_foo()

TypeError: static_foo() missing 1 required positional argument: 'x'

In [41]:
a.static_foo

<function __main__.A.static_foo(x)>